In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

In [2]:
import os
list_csv = sorted(os.listdir("/content/drive/MyDrive/MutualFunds"))

In [3]:
df = pd.read_csv("/content/drive/MyDrive/MutualFunds/Aditya_Birla_Sun_Life_Banking_and_Financial_Services_Fund_-_Direct_Plan_-_Dividend.csv")
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Scheme Code', 'Scheme Name',
       'ISIN Div Payout/ISIN Growth', 'ISIN Div Reinvestment',
       'Net Asset Value', 'Repurchase Price', 'Sale Price', 'Date'],
      dtype='object')

In [ ]:
mf_data = []

for csv in list_csv:
  path = "/content/drive/MyDrive/MutualFunds/"+csv
  df = pd.read_csv(path)
  df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

  
  df = df.rename(columns = {'Sale Price':'Sale_Price'})
  df = df.rename(columns = {'Net Asset Value':'NAV'})
  df = df.rename(columns = {'Repurchase Price':'RP'})

  df.Date = pd.to_datetime(df.Date, format="%Y-%m-%d")
  df = df.set_index("Date", drop=False)
  df.reset_index(drop=True, inplace=True)

  df = df[df.Date <= "2018-06-29"]

  # print(df.dtypes)
  
  df['RP'] = df['RP'].astype("float")
  df['NAV'] = df['NAV'].astype("float")
  df['Sale_Price'] = df['Sale_Price'].astype("float")
  df.fillna(df.mean(),inplace=True)

  mf_data.append(df)

In [5]:
print(mf_data[0].head())

   Scheme Code                                        Scheme Name  \
0       125598  Aditya Birla Sun Life Banking and Financial Se...   
1       125598  Aditya Birla Sun Life Banking and Financial Se...   
2       125598  Aditya Birla Sun Life Banking and Financial Se...   
3       125598  Aditya Birla Sun Life Banking and Financial Se...   
4       125598  Aditya Birla Sun Life Banking and Financial Se...   

  ISIN Div Payout/ISIN Growth ISIN Div Reinvestment    NAV     RP  Sale_Price  \
0                INF209K013W3          INF209K015W8  14.22  13.94       14.22   
1                INF209K013W3          INF209K015W8  14.43  14.14       14.43   
2                INF209K013W3          INF209K015W8  14.41  14.12       14.41   
3                INF209K013W3          INF209K015W8  13.97  13.69       13.97   
4                INF209K013W3          INF209K015W8  13.97  13.69       13.97   

        Date  
0 2015-01-01  
1 2015-01-02  
2 2015-01-05  
3 2015-01-06  
4 2015-01-07  


In [ ]:
avg_mf = []
cols = ['NAV', 'RP','Sale_Price']
for each_stock in mf_data:
    # avg_df = pd.Dataframe()
    
    each_stock = each_stock.drop(['Scheme Code', 'Scheme Name',
       'ISIN Div Payout/ISIN Growth', 'ISIN Div Reinvestment','Date'], axis =1)

    nav_avg = each_stock.loc[:, 'NAV'].mean()
    rp_avg = each_stock.loc[:, 'RP'].mean()
    sp_avg = each_stock.loc[:, 'Sale_Price'].mean()
    close_avg = each_stock.loc[:, 'Close'].mean()
    vol_avg = each_stock.loc[:, 'Volume'].mean()

    data = {'NAV':[nav_avg],'RP':[rp_avg],'Sale_Price':[low_avg],'Close':[close_avg],'Volume':[vol_avg]}
    df = pd.DataFrame(data)

    avg_mf.append(df)